# Implementing a transformer for time-series forecasting

https://towardsdatascience.com/how-to-make-a-pytorch-transformer-for-time-series-forecasting-69e073d4061e



In [ ]:
import torch
import torch.nn as nn 
import math
from torch import nn, Tensor

class PositionalEncoder(nn.Module):
    def __init__(self,
                 dropout: float=0.1,
                 max_seq_len: int=5000,
                 d_model: int=512,
                 batch_first: bool=False):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(p=dropout)
        self.batch_first = batch_first
        self.x_dim = 1 if batch_first else 0

        position = torch.arange(max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_seq_len, 1, d_model)

        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[:x.size(self.x_dim)]

        return self.dropout(x)

class TimeSeriesTransformer:
    def __init__(self,
                 input_size,
                 dim_val,
                 dropout_pos_enc,
                 max_seq_len,
                 n_heads):
        self.encoder_input_layer = nn.Linear(
            in_features=input_size,
            out_features=dim_val
        )

        self.positional_encoding_layer = PositionalEncoder(
            d_model=dim_val,
            dropout=dropout_pos_enc,
            max_seq_len=max_seq_len
        )

        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model=dim_val,
            nhead=n_heads,
            batch_first=True)
        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=4, norm=None)
